In [1]:
%%writefile ipynb_map_reduce_cc1_top10.py
"""The classic MapReduce job: count the frequency of words.
"""
from mrjob.job import MRJob
import re

WORD_RE = re.compile(r"[\w']+")
ALPHABET_RE = re.compile(r"[a-zA-Z]")

class MRCC1TOP10(MRJob):
    
    def mapper(self, _, line):
        the_line = WORD_RE.findall(line)
        for key, value in enumerate(the_line):
            if (value == "my" and (key + 1 < len(the_line))):
                yield (value + "," + the_line[key+1].lower(), 1)

    def combiner(self, value, counts):
        yield (value, sum(counts))

    def reducer(self, value, counts):
        yield (value, sum(counts))
        

if __name__ == '__main__':
    MRCC1TOP10.run()


Overwriting ipynb_map_reduce_cc1_top10.py


In [2]:
!python3 ipynb_map_reduce_cc1_top10.py -r local ./data/shortjokes.csv --output-dir=ipynb_word_count_top10_out --no-output

No configs found; falling back on auto-configuration
No configs specified for local runner
Creating temp directory /var/folders/3g/kpbx6md97b74_076lhbnbvt00000gn/T/ipynb_map_reduce_cc1_top10.jalal.20190413.172554.553678
Running step 1 of 1...
job output is in ipynb_word_count_top10_out
Removing temp directory /var/folders/3g/kpbx6md97b74_076lhbnbvt00000gn/T/ipynb_map_reduce_cc1_top10.jalal.20190413.172554.553678...
